In [1]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import mean_squared_error

import pandas as pd
import os

In [2]:

import statsmodels.api as sm






In [3]:
train_data= pd.read_excel("all_past_year_paramater_include_yeild.xlsx")
train_data.drop(columns=["Unnamed: 0"],inplace=True)

In [4]:
data_yield = pd.read_csv("WheatYield_reset.csv")
#將字串轉成浮點數
for j in range(len(data_yield["Value (bu)"])):
    n = data_yield["Value (bu)"][j].split(",")
    number = ''
    for k in n:
        number += k
        data_yield["Value (bu)"][j] = float(number)

C:\Users\nancy\AppData\Local\Temp\ipykernel_5676\1444070986.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_yield["Value (bu)"][j] = float(number)


In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split

data_true = data_yield[["Value (bu)", "encode_state"]].copy()
train_data_with_dummy = pd.get_dummies(train_data, columns=["Encode"])
# 將特徵和目標變數分開

In [26]:
endog=data_yield["Value (bu)"][:int(round(0.8*(len(data_true)),0))].to_numpy(dtype=float)

In [23]:
exog_v=train_data_with_dummy[:int(round(0.8*(len(train_data)),0))].to_numpy(dtype=float)

In [24]:
endog.drop(columns=["Encode"],inplace=True)


In [25]:
endog.drop(columns=["Value (bu)"],inplace=True)

In [60]:
# 擬合 ARIMAX 模型
model = sm.tsa.ARIMA(endog, exog=exog_v, order=(12, 1, 12))
model_fit = model.fit()

c:\Users\nancy\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
c:\Users\nancy\AppData\Local\Programs\Python\Python311\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:978: UserWarning: Non-invertible starting MA parameters found. Using zeros as starting parameters.
  warn('Non-invertible starting MA parameters found.'


In [48]:
exog_forecast =train_data_with_dummy[-int(round(0.2*(len(train_data)),0)):].to_numpy(dtype=float)


In [ ]:
endog_forecast =data_yield["Value (bu)"][-int(round(0.2*(len(data_true)),0)):].to_numpy(dtype=float)


In [ ]:
endog.drop(columns=["Encode"],inplace=True)
endog.drop(columns=["Value (bu)"],inplace=True)

In [57]:
# 預測未來值
#exog_forecast = train_data_with_dummy 
forecast = model_fit.predict(start=int(round(0.8*(len(data_true)),0)), end=int(round(0.8*(len(data_true)),0))+len(exog_forecast)-1, exog=exog_forecast)



In [35]:
int(round(0.8*(len(data_true))))

914

In [50]:
len(exog_forecast)

229

In [51]:
len(forecast)

229

In [58]:
y_test=data_yield["Value (bu)"][-int(round(0.2*(len(data_true)),0)):].to_numpy(dtype=float)

In [53]:
len(y_test)

229

In [54]:
# Creating a Function for MAPE
import numpy as np

def mape(test, pred):
    test, pred = np.array(test), np.array(pred)
    mape = np.mean(np.abs((test - pred) / test))
    return mape

In [59]:
rmse = mean_squared_error(y_test, forecast, squared=False)

mape_r=mape(y_test,forecast)

print('RMSE:', rmse)
print('MAPE:', mape_r)
r2 = 1 - ((forecast - y_test) ** 2).sum() / ((y_test - y_test.mean()) ** 2).sum()

print('R-squared:', r2)

RMSE: 43916019.10095362
MAPE: 2.4443177163785363
R-squared: -0.8163593475962712
